Od tabule k počítači
====================

***Cílem úvodní části je ukázat, co se změní, pokud přestaneme výpočty dělat ručně a zkusíme si práci ulehčit využitím počítače.***

In [ ]:
# Nutné importy pro celý notebook
import time
import math
import decimal
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Integrál

Začněme typickým příkladem z matematické analýzy:

$$
\int_0^1\, x^{30}e^{x-1}\,\mathrm{d}x
$$


## Analýza

In [ ]:
# TODO: Jaké vlastnosti má zadáná funkce? Dokážeme říct i něco o hodnotě integrálu?
# TODO: Vykreslete funkci užitím knihovny matplotlib

## Analytické řešení

Integrál ze zadání je typickým představitelem příkladu pro řešení metodou *per partes*.

Zkusme si náš příklad zobecnit a označme náš integrál jako $I$, původní zadání pak odpovídá výpočtu $I_{30}$:

$$
I_n = \int_0^1\, x^{n}e^{x-1}\,\mathrm{d}x
$$

Po jednom použití metody *per partes* získáváme:

$$
\begin{aligned}
I_n &= [x^ne^{x-1}]^1_0 - n \int_0^1\, x^{n - 1}e^{x-1}\,\mathrm{d}x\\
I_n &= 1 - n \int_0^1\, x^{n - 1}e^{x-1}\,\mathrm{d}x
\end{aligned}
$$

Pro $n = 30$ stačí použít metodu *per partes* ještě 29krát a máme výsledek!

## Rekurentní řešení (1)

Nebo si můžeme všimnout, že integrál na pravé straně výrazu vypadá přesně jako $I_{n - 1}$. Můžeme tedy snadno získat následující rekurentní vztah:

$$
I_n = 1 - nI_{n - 1}
$$

Snadno jsme navíc schopni spočítat $I_0$:

$$
I_0 = \int_0^1\, x^0e^{x-1}\,\mathrm{d}x = [e^{x-1}]^1_0 = e^0 - e^{-1} = \mathbf{1 - 1 / e}
$$

Nyní kombinací předchozích můžeme už snadno spočítat hodnotu libovolného $I_n$, tedy např. $I_{30}$. Nejprve inicializace:

In [ ]:
# TODO: doplňte kód pro výpočet I[0]

A nyní výpočet $I_{30}$:

In [ ]:
# TODO: Doplňte kód pro výpočet I[1] až I[30]

Podívejme se nyní na kompletní průběh výpočtu:

In [ ]:
for i in range(31):
    print(f'I[{i:2d}] = {I[i]}')

# Reprezentace čísel v počítači

Při přemýšlení o reprezentaci čísel v počítači má smysl rozdělit si je na dvě kategorie:

- celá čísla
- reálná čísla

Jednu věc ale mají společnou. Jak celých, tak reálných čísel je nekonečně mnoho. Abychom mohli nějaké číslo v počítači uložit, musíme mít k dispozici dostatečně velkou paměť. Velikost paměti je ale zcela jistě konečná (a tedy omezená).

## Reprezentace celých čísel

V desítkové soustavě je pro nás přirozené rozepsat číslo 459 jako:
$459 = 4\cdot 10^2 + 5\cdot 10^1 + 9\cdot 10^0 $

Ve dvojkové soustavě je princip zcela analogický, jen máme k dispozici jen číslice 0 a 1 a základ je nyní 2. Např:

$(1001)_2 = 1\cdot 2^3 + 0\cdot 2^2 + 0\cdot 2^1 + 1\cdot 2^0 = 8 + 1 = 9$

Zkusme nyní napsat jednoduchou funkci, která převede číslo z libovolné soustavy (s číslicemi 0-9) do desítkové:

In [ ]:
# TODO: Napište jednoduchou funkci, která provede tuto konverzi
def convert(digits: str, base: int) -> int:
    pass

Ale lze samozřejmě použít i efektivnější nebo obecnější varianty, jako je `np.polyval` pro vyhodnocení polynomu.

$a_3x^3 + a_2x^2 + a_1x^1 + a_0x^0$. Stačí zadat pole koeficientů a hodnotu $x$.

Například:

In [ ]:
coeffs = [int(d) for d in '1001']
np.polyval(coeffs, 2)

A nebo samozřejmě použít nepovinný argument `base` funkce `int`:

In [ ]:
int('1001', base=2)

Procesor počítače "nejsnáze" pracuje s reprezentací čísel, pro kterou byl navržen. Například pro celá čísla používá k uložení standardně 4 B, což odpovídá sekvenci 32 nul nebo jedniček.  

### Celá nezáporná čísla

Omezme se nejprve na celá nezáporná čísla (*unsigned integer 32 bit* = `uint32`) a ukažme si, jak jsou interně uložena:

In [ ]:
def print_uint32(x):
    print(f'{x}: {x:032b}')

for i in range(10):
    print_uint32(np.uint32(i))

Je zjevné, že maximální číslo, která takto dokážeme uložit, je $2 ^ {32} -1$:

In [ ]:
x = np.uint32(2 ** 32 - 1)
print_uint32(x)

Co se stane, pokud bychom zkusili číslo o 1 větší?

In [ ]:
x = np.uint32(2 ** 32)
# Deprecation warning, nahraďte: x = np.array(2 ** 32).astype(np.uint32)

print_uint32(x)

In [ ]:
x = np.uint32(2 ** 32 - 1)
print(x)
print(type(x))
print(x + 1)
print(type(x + 1))

### Celá čísla se znaménkem

Někdy je samozřejmě nutné pracovat i se zápornými čísly. Jejich reprezentace je podobná ale odlišná. Mohlo by se zdát, že stačí vyhradit jeden bit na znaménko, takže bychom pak měli rozsah v kladných i záporných číslech stejný, ale poloviční oproti neznaménkové variantě. Tento způsob má však nevýhodu, protože bychom měli kladnou (+0) ale i zápornou nulu (-0).

Reálně se používá trochu jiná varianta, která obsahuje nulu pouze jednu. Detaily popisovat nebudeme, ale lze je nalézt třeba na [Wikipedii](https://cs.wikipedia.org/wiki/Dvojkov%C3%BD_dopln%C4%9Bk]). Důsledkem toho je asymetrický rozsah:

In [ ]:
for x in (np.int8, np.int16, np.int32, np.int64):
    info = np.iinfo(x)
    print(f'{x}, min = {info.min}, max = {info.max}')

### Typ `int` v Pythonu

Základní typ `int` v Pythonu naopak žádná omezení na velikost nemá a jsme omezeni jen velikostí dostupné paměti:

In [ ]:
2 ** 1000

In [ ]:
import sys
for exp in range(0, 20):
    number = 10 ** exp
    print(f"Velikost čísla {number:.0e}: {sys.getsizeof(number)} B")

Navíc lze využít vektorizované operace (SIMD = Single Instruction Multiple Data):

![SIMD](https://ftp.cvut.cz/kernel/people/geoff/cell/ps3-linux-docs/CellProgrammingTutorial/CellProgrammingTutorial.files/image008.jpg)

Pro 512bitové registry (AVX-512) umíme zpracovat až 64 hodnot `np.int8` najednou.


In [ ]:
size = 100_000_000
python_list = [i for i in range(size)]

start = time.time()
python_result = [x * 2 for x in python_list]
end = time.time()
print(f'Python int: {end - start:.3f} s')


for dtype in [np.int8, np.int16, np.int32, np.int64]:
    numpy_array = np.arange(size, dtype=dtype)
    
    start = time.time()
    numpy_result = numpy_array * 2
    end = time.time()
    print(f'NumPy {dtype}: {end - start:.3f} s')

## Reprezentace reálných čísel

Vraťme se nejdříve opět k teorii a podívejme se, jak rozepsat číslo s desetinnou částí:

$26.185 = 2\cdot 10^1 + 6\cdot 10^0 + 1\cdot 10^{-1} + 8\cdot 10^{-2} + 5\cdot 10^{-3}$

A analogicky pro dvojkovou soustavu:

$(10.101)_2 = 1\cdot 2^1 + 0\cdot 2^0 + 1\cdot 2^{-1} + 0\cdot 2^{-2} + 1\cdot 2^{-3} = 2 + 0 + 1/2 + 0 + 1/8 = 2.625$ 

Reprezentace reálných čísel je výrazně složitější než u celých čísel, ale toto základní schéma platí a má jeden přímý důsledek. V počítači nelze *takto* reprezentovat čísla, která nelze složit jako konečný součet mocnin dvojky, např. i tak jednoduché číslo jako 0.1:

In [ ]:
print(f'{0.1:.100f}')

Pozn. Zde pro úplnost jen dodejme, že binární zápis čísla 0.1 je nekonečný periodický, $0.1 = (0.0\overline{0011})_2$.

Srovnejme s mnohem "ošklivějším" číslem v desítkové soustavě 0.0009765625, které ovšem odpovídá $2 ^{-10}$, a bude tedy uloženo přesně:

In [ ]:
print(f'{0.0009765625:.100f}')

Standardně Python vypisuje pouze prvních několik desetinných míst, takže si toho nemusíme vůbec všimnou, ale už některé základní operace nemusí fungovat zcela intuitivně. Srovnejme:

In [ ]:
print(0.1 + 0.2 == 0.3)
print(0.5 + 0.25 == 0.75)

Vzhledem k vysvětlení výše by nás výsledek ale neměl překvapit. Nepřesná aritmetika reálných čísel je implicitní vlastnost, o které je potřeba vědět a umět s ní nakládat.

Jak tedy správně porovnávat reálná čísla? Musíme si nastavit limit $\epsilon$, pod který považujete (pro danou aplikaci) dvě čísla za stejná.

$a \stackrel{def}{=} b \leftrightarrow |a - b| < \epsilon$

Výše uvedené porovnání tedy lze nahradit takto:


In [ ]:
x = 0.1
y = 0.2
a = x + y
b = 0.3

eps = 10e-6

abs(a - b) < eps

Zde by se mohlo zdát, že dvojková soustava je oproti "naší" desítkové velmi nepraktická. Ale ta má problémy úplně stejné. Jak bychom zapsali například jednu třetinu? Její zápis v desítkové soustavě obsahuje nekonečně mnoho cifer (0.333...). Ale v trojkové soustavě je zápis naopak velmi jednoduchý:

$1/3 = 3^{-1} = (0.1)_3$


## Typy pro reálná čísla v Pythonu

Oproti celočíselným typům, kde byl rozdíl mezi standardním typem `int` a typy poskytovanými knihovnou NumPy, je u reálných čísel situace jednodušší.

Typ `float` vlastnostmi odpovídá typu `np.float64`, dále NumPy ještě nabízí `np.float16`, `np.float32` a `np.float128`. Tyto typy se liší (z definice) množstvím paměti a s tím související přesností. V příkladech výše jsme tedy implicitně používali reprezentaci na 8 bytech.

Tyto typy umí oproti celočíselným navíc i reprezentovat speciální hodnoty, jako je $\pm\infty, \pm 0$ a NaN (= Not a Number).

In [ ]:
print(1 / np.inf)
print(1 / -np.inf)
print(np.float32(1) / 0)
print(-np.float32(1) / 0)
print(np.sqrt(-1))

Jaké bude chování, pokud použijeme typ `float` ze standardního Pythonu a/nebo knihovnu `math`? Vyzkoušejte.

Zmínili jsme, že velikost typu souvisí s přesností. Jak se tedy reprezentují reálná čísla v počítači? Ukažme si zjednodušený popis, který se však od toho skutečného liší už jen v technických detailech. Pro jednoduchost si princip ukažme v desítkové soustavě.

Uvažme např. dvě čísla: 12.04 a -0.0023. Obě můžeme přepsat do tzv. vědecké notace (znáte z kalkulaček):


$$
\begin{aligned}
12.04 &= +1.204 \cdot 10^1\\
-0.0023 &= -2.3 \cdot 10^{-3}
\end{aligned}
$$


V Pythonu analogicky můžeme použít formátovací styl `e`:

In [ ]:
print(f'{12.04:e}')
print(f'{-0.0023:e}')

Reálně se však samozřejmě pro mantisu i exponent používá dvojková soustava.

Musíme tedy vždy uložit znaménko (1 bit) a poté dle zvoleného typu máme určen počet bitů pro mantisu a exponent. Pro typ `np.float32` je to 23 bitů pro mantisu a 8 bitů pro exponent. Celkem 1 + 23 + 8 = 32 bitů = 4 B. Zjednodušeně řečeno, velikost mantisy určuje, kolik prvních číslic budeme mít reprezentováno správně a velikost exponentu udává, jak velká, resp. malá čísla umíme reprezentovat.

Ukažme si nyní pár příkladů, kde budeme používat `np.float32`. Všechny zmíněné vlastnosti platí i pro ostatní reálné typy, liší se jen počtem bitů pro mantisu a exponent.

In [ ]:
print(np.float32(12.56))
print(np.float32(11.19e15))
print(np.float32(-0.000_000_000_000_12))
print(np.float32(123_456_789))

### Absolutní a relativní chyba

První tři příklady zobrazují očekáváné chování (byť víme, že žádné z těchto čísel nebude uloženo přesně), poslední příklad však ukazuje limit velikosti mantisy. Pro typ `np.float32` se udává, že umí zachovat zhruba prvních 6 dekadických číslic. Pozor, tento příklad ukazuje, že u přesnosti nejde nutně o desetinnou část. Jaká je velikost chyby?

Rozlišme *absolutní* ($E_a$) a *relativní chybu* ($E_r$):

$$
\begin{aligned}
E_a &= |\overline{x} - x|\\
E_r &= \frac{E_a}{x} = \frac{|\overline{x} - x|}{x}
\end{aligned}
$$

kde $x$ je přesná hodnota a $\overline{x}$ její odhad.

V našem příkladě tedy chyba vzniklá uložením toho čísla:

In [ ]:
print(f'Absolutní chyba: {abs(123_456_790 - 123_456_789)}')
print(f'Relativní chyba: {abs(123_456_790 - 123_456_789) / 123_456_789}')

Relativní velikost chyby reprezentace je v tomto případě poměrně malá. Pro úplnost dodejme, že uložení stejného čísla v celočíselném typu stejné velikosti by žádnou chybu nepřineslo.

### Operace s reálnými typy

Zatím jsme se pouze zaměřovali na uložení reálných čísel, ale většinou chceme s čísly dělat i nějaké další operace. Ukažme si nejjednodušší z nich, sčítání na příkladu, kdy se obě čísla řádově liší:

In [ ]:
x = np.float32(1e7)
y = np.float32(0.5)
print(f'{x + y:10f}')

Ačkoliv zjevně nedostáváme matematicky správný výsledek, v rámci zvolené přesnosti je korektní. Porovnejme ovšem následující výrazy:

In [ ]:
(x + y) + y, x + (y + y)

Tento příklad demonstruje jednu nepříjemnou vlastnost. **Sčítání reálných čísel v počítači není asociativní**. Záleží tedy na pořadí.

Jak se s tím vypořádat? Přímočaré řešení je seřadit čísla dle jejich velikosti v absolutní hodnotě a pak sečíst, díky čemuž se nám mohou potenciálně menší čísla sečíst do větších, které už se ve výsledku projeví. Takový postup je ovšem pomalejší. Připomeňme si, že složitost řazení je ${\mathcal O}(n\log n)$, zatímco prostý součet je lineární, tedy ${\mathcal O}(n)$.

Srovnejme nyní pár následujích řešení, včetně naivního součtu:

In [ ]:
# TODO: Naprogramujte naivní součet (bez použití sum, pouze s for cyklem)
def naive_sum(array):
    pass

In [ ]:
a = [x, y, y]

In [ ]:
print(sum(a))
print(math.fsum(a))
print(np.sum(a))
print(naive_sum(a))

Ačkoliv všechny funkce implementují součet, každá používá jiný algoritmus, který se liší velikostí chyby a rychlostí. Detaily lze najít v dokumentaci. Ukažme si v následujícím příkladu větší srovnání (ChatGPT):

In [ ]:
size = 10 ** 6

mean = 0.0
std_dev = 1.0
data_list_float64 = np.random.normal(mean, std_dev, size).tolist()
data_list_float32 = [np.float32(x) for x in data_list_float64]
data_array_float64 = np.array(data_list_float64, dtype=np.float64)
data_array_float32 = np.array(data_list_float32, dtype=np.float32)

def time_function(func, data):
    start = time.time()
    result = func(data)
    end = time.time()
    return result, end - start

def naive_sum(data):
    if isinstance(data[0], np.float32):
        total = np.float32(0.0)
    else:
        total = 0
    for value in data:
        total += value
    return total

results = []
errors = []
data_types = [
    ('list[float64]', data_list_float64),
    ('list[float32]', data_list_float32),
    ('array[float64]', data_array_float64),
    ('array[float32]', data_array_float32),
]
functions = [
    ('naive_sum', naive_sum),
    ('sum', sum),
    ('math.fsum', math.fsum),
    ('np.sum', np.sum),
]

reference_value, _ = time_function(math.fsum, data_list_float64)

for dtype_label, data in data_types:
    for func_label, func in functions:
        result, time_taken = time_function(func, data)
        relative_error = abs((result - reference_value) / reference_value) if reference_value != 0 else 0
        results.append({'Typ': dtype_label, 'Funkce': func_label, 'Čas (s)': time_taken})
        errors.append({'Typ': dtype_label, 'Funkce': func_label, 'Relativní chyba': relative_error})

results_df = pd.DataFrame(results)
results_df = results_df.pivot(index='Funkce', columns='Typ', values='Čas (s)')

errors_df = pd.DataFrame(errors)
errors_df = errors_df.pivot(index='Funkce', columns='Typ', values='Relativní chyba')

reference_time = results_df.loc['sum', 'list[float64]']
normalized_results_df = results_df / reference_time

print('Normalizavaný čas (relativně k \'sum\' na \'list[float64]\'):')
print(normalized_results_df)
print()
print('Relativní chyby (ve srovnání s \'math.fsum\' na \'list[float64]\'):')
print(errors_df)

Dle očekávání se ukazuje, že `np.sum` je výrazně rychlejší při práci s NumPy poli (SIMD). Naopak ostatní funkce pracují lépe se standardními seznamy (`list`) z Pythonu. Nejvyšší přesnosti dosahuje `math.fsum`, ale s výjimkou naivního řešení dávají pro oba typy ostatní funkce dobré výsledky. Naivní přístup je vhodný pouze pro edukativní účely.



## Limity velikosti reálných typů

V předchozích příkladech jsme se zaměřovali hlavně na mantisu, nyní se podívejme, jakou roli hraje velikost exponentu.

In [ ]:
finfo_float32 = np.finfo(np.float32)
print("Největší hodnota:", finfo_float32.max)
print("Nejmenší hodnota:", finfo_float32.min)
print("Nejmenší kladné číslo:", finfo_float32.tiny)

Ukažme si příklad, kde může hrát limit na velikost čísla roli. Podívejme se na výpočet geometrického průměru.

$$
G(x_1,\ldots, x_n) = \sqrt[n]{\prod_{i = 1}^n x_i}
$$

Zkusme nyní spočítat geometrický průměr 100 náhodných čísel mezi 0 a 10:

In [ ]:
N = 100
array_big = np.random.sample(N).astype(np.float32) * 10

In [ ]:
# TODO: Doplňte kód pro výpočet geometrického průměru užitím standardních funkcí NumPy a použijte jej na array_big

def gmean(array):
    pass

gmean(array_big)

Vyzkoušejme stejný algoritmus pro čísla mezi 0 a 0.1:

In [ ]:
array_small = np.random.sample(N).astype(np.float32) / 10
print(np.any(array_small == 0))
gmean(array_small)

Jak tedy spočítat geometrický průměr? Může nám pomoci logaritmická transformace, která převádí násobení na sčítání. Vzpomeňme si, že platí tyto vztahy:

$$
\begin{aligned}
e^{\ln(x)} &= x\\
\ln{x^k} &= k\ln{x}\\
\ln{xy} &= \ln{x} + \ln{y}
\end{aligned} 
$$

Odtud můžeme náš vzorec pro geometrický průměr přepsat na následující verzi:

$$
G(x_1,\ldots, x_n) = \sqrt[n]{\prod_{i = 1}^n x_i} = \prod_{i = 1}^n (x_i)^{1/n} = \exp{\left(\prod_{i = 1}^n \ln(x_i)^{1/n}\right)} = \exp{\left(\frac{1}{n}\sum_{i=1}^n \ln{x_i}\right)}
$$

V kódu pak:

In [ ]:
#TODO Napište funkci gmean, která spočítá geometrický průměr dle vzorce výše:
def gmean_log(array):
    pass

Vyzkoušejme nyní pro oba naše příklady:

In [ ]:
print(gmean_log(array_big))
print(gmean(array_small))

Stejného výsledku pak docílíme využitím vhodné funkce, zde nám pomůže knihovny SciPy:

In [ ]:
print(scipy.stats.gmean(array_big))
print(scipy.stats.gmean(array_small))

Nyní, když víme, jak se pracuje s reálnými čísly v počítači a jaká úskalí to přináší, můžeme se vrátit zpátky k úvodnímu příkladu.

### Integrál - analýza chyby

Připomeňme si, že jsme pro výpočet $I_n$ používali rekurentní vzorec:

$$I_n = 1 - nI_{n - 1}$$

Zkusme si nyní udělat tento rozvoj pro více členů:

$$I_n = 1 - nI_{n - 1} = 1 - n[1 - (n-1)I_{n - 2}] = 1 - n\{1 - (n-1)[1 - (n - 2)I_{n - 3}]\}$$

Náš výpočet začíná s hodnoutou $I_0 = 1 - 1/e$. Už víme, že toto číslo bude uloženo s nějakou, byť velmi malou chybou na vstupu. Dodejme navíc, že jsme implicitně používali typ `float`, který má přesnost na zhruba 16 desítkových číslic.

$I_n$ však ve svém výpočtu obsahuje člen $n!\cdot I_0$. Kolik je $30!$?

In [ ]:
# TODO: Jak spočítat snadno faktoriál zadaného čísla?

Při násobení takto velkým číslem už může byť velmi malá chyba na začátku dominovat výsledku. Pomohl by nám větší typ `np.float128`?

In [ ]:
# TODO Nahraďte kód z úvodu, aby obsahoval jako typ np.float128

### Rekurentní řešení (2)

Správné řešení zde spočívá v otočení rekurentního vztahu:

$$
\begin{aligned}
I_n &= 1 - nI_{n - 1}\\
I_{n - 1} &= \frac{1 - I_n}{n} 
\end{aligned}
$$

Nyní ale pro výpočet $I_{30}$ potřebujeme ale znát hodnotu $I_{31}$. Zjevně z charakteru původní funkce $x^ne^{x-1}$ bude větší $n$ na intervalu $[0, 1]$ znamenat nižší hodnoty funkce. Zkusme tedy zvolit např. $I_{35} = 0$.

In [ ]:
I = np.empty(36, dtype=np.float64)
I[35] = 0
for n in reversed(range(30, 36)):
    I[n - 1] = (1 - I[n]) / n

print(I[30])

Proč je toto řešení vhodnější? V každém kroku se zde totiž chyba na vstupu dělí, nikoliv násobí jako v předešlém způsobu. Samozřejmě nejlepším řešením bude použít nějakou existující funkci pro numerickou integraci, zde např. z knihovny SciPy: 

In [ ]:
result = scipy.integrate.quad(lambda x: x ** 30 * np.exp(x - 1), 0, 1)[0]
print(result)

Srovnejme tento výsledek s výsledkem dosaženým naším postupem, rozdíl je zde zanedbatelný:

In [ ]:
print(abs(result - I[30]))

Výsledek je o to zajímavější, že zatímco v první variantě byla chyba na vstupu menší než $10^{-16}$, ve druhé variantě, kdy jsme začínali s $I_{35} = 0$ je chyba výrazně větší (viz výpočet níže). Přesto jsme dostali správný výsledek.

In [ ]:
scipy.integrate.quad(lambda x: x ** 35 * np.exp(x - 1), 0, 1)[0]

Závěrem dodejme, že lze použít třeba i jednoduchou lichoběžníkovou metodu, kterou znáte z matematiky, přímo z NumPy.

![Lichoběžníková metoda](https://www.bragitoff.com/wp-content/uploads/2015/08/TrapezoidRule1.png)

Přesnost výsledku můžeme pak ovlivnit počtem dělicích bodů intervalu:

In [ ]:
N = 100
xs = np.linspace(0, 1, N)
np.trapz(xs ** 30 * np.exp(xs - 1), xs)

## Dodatek - přesné výpočty v Pythonu

Pokud bychom skutečně potřebovali reprezentovat reálná čísla přesně, nabízí k tomu Python modul `decimal`. Vyzkoušejme:

In [ ]:
decimal.Decimal('0.1') + decimal.Decimal('0.2') == decimal.Decimal('0.3')

Daní bude ale samozřejmě množství paměti a rychlost operací. Přesnost lze explicitně nastavit:

In [ ]:
init = decimal.getcontext().prec
print(f'Původní přesnost: {init}')
for prec in [init, 5, 100]:
    decimal.getcontext().prec = prec
    x = decimal.Decimal(1) / 3    
    print(f'Hodnota: {x}')
    print(f'Velikost: {sys.getsizeof(x)}')

# Nastavme zpátky iniciální hodnotu
decimal.getcontext().prec = init

In [ ]:
array = np.random.sample(10 ** 6)
decimal_array = [decimal.Decimal(x) for x in array]

In [ ]:
sum(decimal_array), np.sum(array)

In [ ]:
%%timeit
sum(decimal_array)

In [ ]:
%%timeit
np.sum(array)